In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dat
import matplotlib as mplt
import datetime as dt
%matplotlib inline
sns.set()

In [2]:
auctions = pd.read_csv("data/auctions.csv", dtype={'date': 'str','device_id': 'category', 'ref_type_id': 'category','source_id':'category'}, parse_dates = ['date'])
auctions.head(5)

,date,device_id,ref_type_id,source_id
0,2019-04-23 18:58:00.842116,2564673204772915246,1,0
1,2019-04-23 18:58:01.530771,4441121667607578179,7,0
2,2019-04-23 18:58:01.767562,7721769811471055264,1,0
3,2019-04-23 18:58:02.363468,6416039086842158968,1,0
4,2019-04-23 18:58:02.397559,1258642015983312729,1,0


In [3]:
auctions=auctions.sort_values(by='date')

In [7]:
serie=auctions.groupby('device_id').cumcount()
serie=serie.to_list()
auctions['entradas_previas']=serie

In [9]:
fecha_minima=pd.to_datetime('2019-04-18 00:00:00')
fecha_minima
fecha_tope=pd.to_datetime('2019-04-21 00:00:00')
auctions_ventana1=auctions.loc[auctions['date']<fecha_tope].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de dias
auctions_ventana1['tiempo_hasta_aparicion']=(auctions_ventana1['date']-fecha_minima)/ np.timedelta64(1, 's')

In [10]:
auctions_ventana1_reducido=auctions_ventana1.groupby(['device_id']).agg({"date" : min,\
                                                                         #"std_hour" : "first",\
                                                                         #"mean_hour" : "first",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min})\
                                                                            .reset_index('device_id')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
auctions_ventana1_reducido=auctions_ventana1_reducido.dropna()

In [11]:
fecha_minima=pd.to_datetime('2019-04-21 00:00:00')
fecha_tope=pd.to_datetime('2019-04-24 00:00:00')

auctions_ventana2=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de dias
auctions_ventana2['tiempo_hasta_aparicion']=(auctions_ventana2['date']-fecha_minima)/ np.timedelta64(1, 's')

In [12]:
auctions_ventana2_reducido=auctions_ventana2.groupby(['device_id']).agg({"date" : min,\
                                                                         #"std_hour" : "first",\
                                                                         #"mean_hour" : "first",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min})\
                                                                            .reset_index('device_id')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
auctions_ventana2_reducido=auctions_ventana2_reducido.dropna()

In [13]:
fecha_minima=pd.to_datetime('2019-04-22 00:00:00')
fecha_tope=pd.to_datetime('2019-04-25 00:00:00')

auctions_ventana3=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de dias
auctions_ventana3['tiempo_hasta_aparicion']=(auctions_ventana2['date']-fecha_minima)/ np.timedelta64(24, 'h')
auctions_ventana3['dia_mean']=auctions_ventana3['date'].dt.day
auctions_ventana3['dia_desvio_std']=auctions_ventana3['date'].dt.day
auctions_ventana3.head(25)
auctions_ventana3_reducido=auctions_ventana3.groupby(['device_id']).agg({"date" : min,\
                                                                         #"std_hour" : "first",\
                                                                         #"mean_hour" : "first",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min})\
                                                                            .reset_index('device_id')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
auctions_ventana3_reducido=auctions_ventana3_reducido.dropna()

In [14]:
fecha_minima=pd.to_datetime('2019-04-23 00:00:00')
fecha_tope=pd.to_datetime('2019-04-26 00:00:00')

auctions_ventana4=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de dias
auctions_ventana4['tiempo_hasta_aparicion']=(auctions_ventana2['date']-fecha_minima)/ np.timedelta64(24, 'h')
auctions_ventana4['dia_mean']=auctions_ventana4['date'].dt.day
auctions_ventana4['dia_desvio_std']=auctions_ventana4['date'].dt.day
auctions_ventana4.head(25)
auctions_ventana4_reducido=auctions_ventana4.groupby(['device_id']).agg({"date" : min,\
                                                                        # "std_hour" : "first",\
                                                                        # "mean_hour" : "first",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min})\
                                                                            .reset_index('device_id')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
auctions_ventana4_reducido=auctions_ventana4_reducido.dropna()

In [16]:
fecha_minima=pd.to_datetime('2019-04-24 00:00:00')
fecha_tope=pd.to_datetime('2019-04-27 00:00:00')

auctions_ventana5=auctions.loc[((auctions['date']>fecha_minima) & (auctions['date']<fecha_tope) )].copy(deep=False)
#la division es para pasar la diferencia de tiempso de timedelta a float de dias
auctions_ventana5['tiempo_hasta_aparicion']=(auctions_ventana2['date']-fecha_minima)/ np.timedelta64(24, 'h')
auctions_ventana5['dia_mean']=auctions_ventana5['date'].dt.day
auctions_ventana5['dia_desvio_std']=auctions_ventana5['date'].dt.day
auctions_ventana5.head(25)
auctions_ventana5_reducido=auctions_ventana1.groupby(['device_id']).agg({"date" : min,\
                                                                         #"std_hour" : "first",\
                                                                         #"mean_hour" : "first",\
                                                                         'tiempo_hasta_aparicion':"min",\
                                                                        'entradas_previas':min})\
                                                                            .reset_index('device_id')
#entradas previas no se debe usar para predecir en esta ventana, sino para                
auctions_ventana5_reducido=auctions_ventana5_reducido.dropna()